In [1]:
import h5py
import numpy as np
from scipy.io import loadmat
import tensorflow as tf
import threading
import iio

In [2]:
tmp = h5py.File('./deepsea_train/train.mat', 'r')
train_input = tmp['trainxdata']
train_target = tmp['traindata']

tmp = loadmat('./deepsea_train/valid.mat')
valid_input = tmp['validxdata']
valid_target = tmp['validdata']

def test_and_valid_iter(input_, target):
    for i in xrange(input_.shape[0]):
        yield iio.make_example({
            'input': iio.make_float_feature(input_[i].flatten()),
            'target': iio.make_int64_feature(target[i].flatten()),
        })
        
def train_iter(input_, target):
    for i in xrange(input_.shape[2]):
        yield iio.make_example({
            'input': iio.make_float_feature(input_[:,:,i].flatten()),
            'target': iio.make_int64_feature(target[:,i].flatten()),
        })

def write_tfrecords(examples, filename, num_shards=100):
    filenames = iio.sharded_filenames(filename, num_shards)
    writers = map(tf.python_io.TFRecordWriter, filenames)
    for i, example in enumerate(examples):
        t = threading.Thread(
            target=writers[i % num_shards].write,
            args=(example.SerializeToString(),))
        t.daemon = True
        t.start()

In [ ]:
!mkdir -p output

write_tfrecords(
    examples=test_and_valid_iter(valid_input, valid_target),
    filename='./output/valid.tfrecord',
    num_shards=100)

write_tfrecords(
    examples=train_iter(train_input, train_target),
    filename='./output/train.tfrecord',
    num_shards=100)